# TME 5 : Mini-projet Detection de motifs


## Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME4, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faites une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
q = 4

def generateRandomSequences(n, t):
    sequences = []
    i = 0
    j = 0
    while i < t:
        seq = ""
        while j < n:
            seq = seq+nuc[random.randint(0,3)]
            j += 1
        sequences.append(seq)
        i += 1
        j = 0
   
    return sequences

# RENVOIE UNE LISTE DE LISTE DE MOTIFS...
def generateMotifs(k, nM):
    return [generateRandomSequences(k, True) for rs in range(nM)]

def gen_n_substitMotifs(v, k, t):
    motifs = generateMotifs(k, t)
    motifs_res = []
    for motif in motifs:
        lettre_motif = list(motif[0])
        j = 0
        while j < v:
            lettre_motif[random.randint(0, k - 1)] = nuc[random.randint(0,3)]
            motif_add = ''.join(lettre_motif)
            motifs_res.append(motif_add)
            j+=1
    return motifs_res
    
def implantMotifVar(k, v, t, n):
    #sequences aleatoires
    sequences = generateRandomSequences(n, t)
    motifs = gen_n_substitMotifs(v, k, t)
    modified_sequences = []
    i = 0
    while i < t:
        seq = sequences[i]
        motif = motifs[i]
        rand_pos = random.randint(0,len(seq))
        seq = seq[0:rand_pos] + motif + seq[rand_pos+1:len(seq)]
        modified_sequences.append(seq)
        i+=1
    return modified_sequences

motifs = gen_n_substitMotifs(v, k, t)
sequences = implantMotifVar(k, v, t, n)
print(sequences)


['ACTAATTAAGCCG', 'GCTTGGAACATAT', 'AATGTATAAACAA']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visualise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [2]:
k=8  #taille de motif
v=2  #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

In [3]:
motifs = gen_n_substitMotifs(v, k, t)
sequences = implantMotifVar(k, v, t, n)
print(motifs)
print (sequences)
with open("motifs_generated.txt","w") as file:
    for motif in motifs:
        file.write("{}\n".format(motif))

['CCGGCCCT', 'CCGGCCCT', 'TCTGGTTG', 'TCCGGTTG', 'GCATCCCG', 'GCGTCCCG', 'ATGGTAGT', 'ATGGTAGC', 'GACTATTG', 'GACTATTG', 'AATCAGTA', 'AATCAGTA', 'CATTTTTT', 'CATTTTTT', 'ATAGGAAA', 'ATGGGAAA', 'TCCGTTAT', 'TCTGTTAT', 'ATGCCTGC', 'ATGTCTGC']
['TCGAATTCTCAGTTAGG', 'TATAAATTATCAACTCA', 'GATGCCGGTGACCTGGT', 'TTGACCTGGTAACATCA', 'GAATGCTGCTCCTCCTG', 'CATCGCAACAGCTCCTCC', 'GCAAACGAGAGTGGATT', 'CATGACGTTCAAATGAG', 'CTCACCGGTTCTCCTTG', 'TCGGGGTTCTACTTGAC']


3\. Préparation pour l'algorithme _"Brute Force Motif Search"_. Avant d'implementer  l'algorithme _"Brute Force Motif Search"_, faites une fonction pour calculer le score `score(s, seqs)`, où `s` est un ensemble de position initiales `s`$=(s_1,s_2,...,s_k)$  et `seqs` est une matrice de taille `t x n`  (`t` est le nombre de séquences et `n` la taille de ces séquences), pour tester votre fonction utiliser la matrice et les valeurs ci-dessous (voir aussi les slides du cours).


In [17]:



k = 8  # taille de motif
n = 68 #longuer des sequence
t = 5  #nb de sequences

s=(25,20,2,55,59)

seqs = [
'CCTGATAGACGCTATCTGGCTATCCACGTACATAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATACGTACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAAAGTCCGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTACGTATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTAACGTAGGTC'
]

corresp = {0:"A", 1:"C", 2:"G", 3:"T"}



#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []
    i = 0
    for start in s:
        seq = seqs[i]
        motif = seq[start:start+k]
        motifs.append(motif)
        i+=1
    return motifs

#construct Profiles
def profile(motifs, k, t, q):
    PWM = np.zeros((q, k))
    for motif in motifs:
        i = 0
        while i < len(motif):
            l = motif[i]
            if l == 'A':
                PWM[0][i] += 1
            if l == 'C':
                PWM[1][i] += 1
            if l == 'G':
                PWM[2][i] += 1
            if l == 'T':
                PWM[3][i] += 1
            i += 1
    return PWM


def score(s, seqs):
    sc = 0
    motifs = extractSeqs(s, seqs)
    profile_ = profile(motifs, k, t, q)
    i = 0
    j = 0
    base = ''
    consensus = ""
    while j < len(profile_[i]):
        max_ = 0
        while i < len(profile_):
            if profile_[i][j] > max_:
                max_ = profile_[i][j]
                base = corresp[i]
            i += 1
        sc += max_
        consensus += base
        i = 0
        j += 1
    return consensus, sc

consensus, sc = score(s, seqs)
print(consensus)
print(sc)



ACGTACGT
36.0


4\. Implementer l'algoritme _"Brute Force Motif Search"_ pour chercher des motifs de taille variable. Attention, la complexité de cet algorithme est trop importante, car il enumère toutes les positions intiales possibles, nous allons donc le tester seulement sur de petits jeux de données.

In [18]:
from itertools import product

k=4; v=1; t=3; n=10
#enumeration all initial positions
def initialPosition(k, n, t):
	iniPos = []
	allIniPos = []
	for i in range(0, n-k+1):
		iniPos.append(i)
        
	for subset in product(iniPos, repeat=t):
		allIniPos.append(subset)
	return allIniPos

#Pour tester la generation de positions
#print(initialPosition(k, n, t))

In [19]:
import time
def bruteForceMotifSearch(sequences, t, n, k):
	bestscore = 0; bestMotif=''
	allIniPos = initialPosition(k, n, t)
	for starts in allIniPos:
		consensus, score_act = score(starts, sequences)
		if score_act > bestscore:
			bestscore = score_act
			bestMotif = consensus

	return  bestMotif, bestscore

#test
k=4; v=1; t=3; n=10
motifs = gen_n_substitMotifs(v, k, t)
sequences = implantMotifVar(k, v, t, n)
print (sequences)
start = time.time()
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
end = time.time()
print (bestMotif, bestscore)
print("durée exec bruteForce = {}".format(end - start))

['CATCGTAATTGAG', 'GCATGAGCTCAGC', 'ACACATCAGCCCT']
CATC 11.0
durée exec bruteForce = 0.006499767303466797


5\. L'algorithme est-il performant sur ce jeu de données ? Sinon, pourquoi ?

<font color='white'>
réponse: Oui il semble performant vu que les donnees sont de taille reduite
</font>

6\. Tester l'agorithme avec les paramètres suivants `k=7`, `v=1`, `t=5`, et `n=20`. Avez vous de meilleurs performances ?

In [22]:
k=7; v=1; t=5; n=20

sequences = implantMotifVar(k, v, t, n)
print (sequences)
start = time.time()
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
end = time.time()
print (bestMotif, bestscore)
print("durée exec bruteForce = {}".format(end - start))

# non, c'est bien plus long et c'est logique vu que les donnees sont plus importantes


['GTATGCGCCGAAGCTATTAATCGTCG', 'TCTTAACCTATCGAGACACGTTTGTC', 'CTCAAACTGTTTGGGGCGCTGCTCGT', 'ATAGTACTTCCAGTACGTCTATGCGT', 'CAAGCCGTCTTTGTGGATAACGGAAT']
AAACTAT 26.0
durée exec bruteForce = 19.726464986801147


7\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les mêmes jeux de données que la question 3 (voir slides de cours). Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles.

In [23]:
#generer les k-mers valid
def kmers(sequences, k):
    validkmers = [] # liste qui va contenir une liste des motifs possibles sur la sequence en fonction de la position de depart
    for seq in sequences:
        kmer_seq = []
        i = 0
        while i < (len(seq) - k + 1):
            kmer_seq.append(seq[i: i + k]) # on ajoute chaque motif de taille k
            i += 1
        validkmers.append(kmer_seq) # on ajoute la liste de liste de motifs a la liste que l'on retourne
    return validkmers

validkmers = kmers(seqs, k)
print(validkmers)


[['CCTGATA', 'CTGATAG', 'TGATAGA', 'GATAGAC', 'ATAGACG', 'TAGACGC', 'AGACGCT', 'GACGCTA', 'ACGCTAT', 'CGCTATC', 'GCTATCT', 'CTATCTG', 'TATCTGG', 'ATCTGGC', 'TCTGGCT', 'CTGGCTA', 'TGGCTAT', 'GGCTATC', 'GCTATCC', 'CTATCCA', 'TATCCAC', 'ATCCACG', 'TCCACGT', 'CCACGTA', 'CACGTAC', 'ACGTACA', 'CGTACAT', 'GTACATA', 'TACATAG', 'ACATAGG', 'CATAGGT', 'ATAGGTC', 'TAGGTCC', 'AGGTCCT', 'GGTCCTC', 'GTCCTCT', 'TCCTCTG', 'CCTCTGT', 'CTCTGTG', 'TCTGTGC', 'CTGTGCG', 'TGTGCGA', 'GTGCGAA', 'TGCGAAT', 'GCGAATC', 'CGAATCT', 'GAATCTA', 'AATCTAT', 'ATCTATG', 'TCTATGC', 'CTATGCG', 'TATGCGT', 'ATGCGTT', 'TGCGTTT', 'GCGTTTC', 'CGTTTCC', 'GTTTCCA', 'TTTCCAA', 'TTCCAAC', 'TCCAACC', 'CCAACCA', 'CAACCAT'], ['AGTACTG', 'GTACTGG', 'TACTGGT', 'ACTGGTG', 'CTGGTGT', 'TGGTGTA', 'GGTGTAC', 'GTGTACA', 'TGTACAT', 'GTACATT', 'TACATTT', 'ACATTTG', 'CATTTGA', 'ATTTGAT', 'TTTGATA', 'TTGATAC', 'TGATACG', 'GATACGT', 'ATACGTA', 'TACGTAC', 'ACGTACG', 'CGTACGT', 'GTACGTA', 'TACGTAC', 'ACGTACA', 'CGTACAC', 'GTACACC', 'TACACCG', 'ACACC

In [25]:
#Voir photo jointe pour comprendre hamdist et totaldistance
#hamdist est le nombre de paires de nucleotides qui ne correspondent pas lorsque v et x sont alignes
def hamdist(v, x):
	"""
	hyp : v et x sont de meme taille
	"""
	if len(v) != len(x):
		print("Erreur : v et x de taille diff\n")
		return -1
	v = v.upper() # on remet en maj les string
	x = x.upper()
	diffs = 0
	i = 0
	while i < len(v):
		if v[i] != x[i]: # si deux lettres different a un endroit on incremente diff
			diffs += 1
		i += 1

	return diffs

#totaldistance est la somme des hamdist minimales d'un motif pour chaque sequence 
def totalDistance(motif, sequences, k):
	td = 0
	valid_kmers = kmers(sequences, k)
	for kmer_seq in valid_kmers:
		min_diffs = k + 1 # variable permettant de comparer les hamdist minimales entre chaque kmer, on l'ajoute a td a chaque nouvelle sequence
		diffs = []
		for kmer in kmer_seq: 
			diffs.append(hamdist(motif, kmer)) # on stocke toutes les valeurs dans une liste pour avoir le min a la fin
		if min(diffs) < min_diffs:
			min_diffs = min(diffs) # on recupere le minimum de la liste diffs si il est plus petit que pour les kmers qui commencaient a une position differente
		
		td += min_diffs
				
	return td

def MedianStringSearch(allkmers, sequences, t, n, k):
	bestDistance = 1000
	bestMotif = ""
	motifDist = {} #all motifs found
	for kmer_seq in allkmers: #on parcours l'ensemble des kmers
		for motif in kmer_seq:	#pour récuperer ensuite chaque motif possible
			dist=totalDistance(motif,sequences,k) #on mesure alors la distance sur l'ensemble des séquences
			if dist<bestDistance: #on compare alors cette distance avec la meilleure distance alors obtenue
				bestDistance=dist #on réaffecte les meilleurs résultats si le motif étudié a un meille score
				bestMotif=motif
			motifDist[motif]=dist #on ajoute ensuite le motif et la distance associée au dictionnaire
	return bestMotif, bestDistance, motifDist

#teste, td = 4
k = 8
"""
print(sequences)
td = totalDistance("acgtacgt", sequences, k)
print (td)
"""
validkmers = kmers(seqs, k)
bestMotif, bestDistance, motifDist = MedianStringSearch(validkmers, seqs, t, n, k)
print ("Meilleur motif : "+bestMotif+" distance : "+'{}'.format(bestDistance))


Meilleur motif : ACGTACGT distance : 4


8\. Comparation entre les algorithmes _"Median String Search"_ et "_Brute Force Motif Search_"

a\. Produire des séquences artificielles avec les paramètres suivants  `k=7`, `v=1`, `t=5`, `n=20`. Comparer les performance et le temps de calculs des deux algorithmes.

In [11]:
k=7; v=1; t=5; n=20

sequence_artif = generateRandomSequences(n,t)

start=time.time()
BFBestMotif,BFBestScore=bruteForceMotifSearch(sequence_artif,t,n,k)
end=time.time()
BFTime=end-start

start=time.time()
MSSkmers=kmers(sequence_artif,k)
MSSBestMotif,MSSBestDIstance,MSSMotifDist = MedianStringSearch(MSSkmers,sequence_artif,t,n,k)
end=time.time()
MSSTime=end-start

print("Durée de l'algorithme BruteForce : {}s \nDurée de l'aglorithme MedianStringSearch : {}s".format(BFTime,MSSTime))

Durée de l'algorithme BruteForce : 31.689541578292847s 
Durée de l'aglorithme MedianStringSearch : 0.0s


b\. Quel algorithme peut etre appliqué au probleme de recherche de motifs? Et comment?

<font color='lightblue'>
réponse: L'algorithme MedianStringSearch peut être appliqué au problème de recherche de motifs car il est efficace, pour l'implémenter il suffit d'avoir les sequences que l'ont souhaite analyser
</font>

In [14]:
k=7; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence

genome = "./Sequence_by_Peaks_6.fasta"

sequences   = readFasta(genome, n)

t = len(sequences)
print(t)
start=time.time()
MSSkmers=kmers(sequences,k)
MSSBestMotif,MSSBestDIstance,MSSMotifDist = MedianStringSearch(MSSkmers,sequences,t,n,k)
end=time.time()
MSSTime=end-start


330
